In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [26]:
#Dados do problema
dados_ger = pd.DataFrame({
    'Barra': [1, 1, 2, 2, 3, 3, 3, 5, 5, 4, 4],
    'P_min (MW)': [0] * 11,
    'P_max (MW)': [10, 20, 15, 40, 30, 40, 20, 10, 20, 60, 75],
    'Custo/MW': [5, 15, 6, 7, 6, 15, 25, 10, 15, 30, 18]
})


bus = [f'Barra {bar+1}' for bar in range(6)]

loads = ['Carga 1']
Nc = len(loads)
dem = [250]


ger = [f'Gerador {gerador+1}' for gerador in range(len(dados_ger['Barra']))]
Ng = len(ger)
c_ener = dados_ger['Custo/MW'].tolist()
pmin = [0] * Ng
pmax = [10, 20, 15, 40, 30, 40, 20, 10, 20, 60, 75]

#Localização do Gerador
ger_loc = pd.DataFrame(0, index=ger, columns=bus)
# Preencher a matriz ger_loc com 1 onde o gerador está localizado em uma barra
for idx, row in dados_ger.iterrows():
    gerador = f'Gerador {idx + 1}'  # Nome do índice para cada gerador
    barra = f'Barra {row["Barra"]}'
    ger_loc.at[gerador, barra] = 1

#Localização da Carga
loads_loc = pd.DataFrame(0, index=loads, columns=bus)
loads_loc.loc['Carga 1', 'Barra 6'] = 1

In [27]:
model = ConcreteModel()

#Sets
model.ger = Set(initialize=ger)
model.loads = Set(initialize=loads)
model.bus = Set(initialize=bus)

#Parâmetros
model.pmin = Param(model.ger, initialize={ger[k]: pmin[k] for k in range(Ng)})
model.pmax = Param(model.ger, initialize={ger[k]: pmax[k] for k in range(Ng)})
model.c_ener = Param(model.ger, initialize={ger[k]: c_ener[k] for k in range(Ng)})
model.dem = Param(model.loads, initialize={loads[k]: dem[k] for k in range(Nc)})
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.load_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: loads_loc.loc[l, b], default=0, mutable=False)

#Variáveis
model.prod = Var(model.ger, domain = NonNegativeReals)

In [28]:
#Restrições
def prod_min(model, gerador):
    return model.prod[gerador] >= model.pmin[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.prod[gerador] <= model.pmax[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def balanco(model):
    return sum(model.prod[i] for i in model.ger) == sum(model.dem[j] for j in model.loads)
model.balanco = Constraint(rule=balanco)

In [29]:
#Função Objetivo:
def objective_rule(model):
    return sum(model.c_ener[i]*model.prod[i] for i in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Executar Otimização:
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

In [30]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.prod[gerador].value))

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 3070.0 

Energia gerada do Gerador 1: 10.00 MWh
Energia gerada do Gerador 2: 20.00 MWh
Energia gerada do Gerador 3: 15.00 MWh
Energia gerada do Gerador 4: 40.00 MWh
Energia gerada do Gerador 5: 30.00 MWh
Energia gerada do Gerador 6: 40.00 MWh
Energia gerada do Gerador 7: 0.00 MWh
Energia gerada do Gerador 8: 10.00 MWh
Energia gerada do Gerador 9: 20.00 MWh
Energia gerada do Gerador 10: 0.00 MWh
Energia gerada do Gerador 11: 65.00 MWh
